In [3]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [4]:
# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명 !!!! 반드시 호수 체크!!!!
model_name = "6호모델_{}_k{}_{}".format(freq, num_k, sc)
model_name

'6호모델_Q_k3_new_rank'

In [5]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트_전체시장.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트_전체시장.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트_전체시장.csv", index_col=0)

In [6]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [7]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [8]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [9]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [10]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [11]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액_전체시장"
                                 ,plus_two="CB금액_전체시장", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '유상증자금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액_전체시장+CB금액_전체시장"
                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '특수관계인에대한자금대여금액_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액_전체시장"
                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트_전체시장"
                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송_전체시장"
                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장', '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장',
       '조회공시요구카운트_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
                                 ,plus_two="불성실공시법인지정카운트_전체시장", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '조회공시요구카운트_전체시장', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장"
                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [19]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장"
                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together9["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [20]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장"
                                 ,plus_two="타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together10["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [21]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
                                 ,plus_two="BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장", return_origianl=False)
print(concat_together11["2012-01"].columns.shape)
concat_together11["2012-01"].columns

(14,)


Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [22]:
divide_together = tools.dict_data_divide(concat_together11,divide_what=['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장',
        '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'], divide_into="총자본",
        return_origianl=True)

divide_together["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [23]:
drop_together1 = tools.dict_data_drop(divide_together.copy(), 
                                      drop_col=['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장',
                                                '당기순이익', '매출채권', '매출총이익', '영업이익','영업현금흐름',"판매 및 관리비"], dropna=False)
print(len(drop_together1["2012-01"].columns))
drop_together1["2012-01"].columns

14


Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본', '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [24]:
drop_togehter = tools.dict_data_drop(drop_together1.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장+타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본', '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [25]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=50, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 50/50 [00:03<00:00, 13.20it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 6,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=6)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 36,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=36)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 4,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=4)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 40,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=40)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 5,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=5)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 7,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=7)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 26,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=26)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 16,
  'model': KMeans(max_iter=500, n_c

## 모델기록


In [26]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

847.0
416.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,416.0,600.0,630.0,543.0,485.0,446.0,622.0,484.0,539.0,441.0,...,745.0,666.0,769.0,628.0,588.0,588.0,683.0,775.0,833.0,847.0
1,560.0,442.0,521.0,631.0,596.0,632.0,450.0,534.0,476.0,583.0,...,553.0,583.0,603.0,735.0,682.0,820.0,576.0,596.0,556.0,675.0
2,618.0,562.0,456.0,434.0,524.0,527.0,528.0,598.0,603.0,594.0,...,678.0,750.0,642.0,662.0,773.0,648.0,796.0,685.0,658.0,517.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,...,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [27]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

eval_one_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.085057,0.096552,0.073563,0.009195,0.073563,0.068966,0.000000,0.091954,0.004598,0.091954,...,0.188506,0.006897,0.186207,0.022989,0.163218,0.016092,0.006897,0.018391,0.013793,0.112644
1,0.020690,0.027586,0.000000,0.006897,0.004598,0.006897,0.085057,0.002299,0.091954,0.009195,...,0.006897,0.011494,0.020690,0.013793,0.011494,0.165517,0.163218,0.154023,0.131034,0.018391
2,0.016092,0.004598,0.018391,0.066667,0.002299,0.000000,0.002299,0.002299,0.002299,0.006897,...,0.016092,0.186207,0.020690,0.190805,0.020690,0.018391,0.025287,0.006897,0.018391,0.018391
포착군집,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,2.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [28]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))
eval_two_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.050633,0.053797,0.034810,0.006329,0.037975,0.050633,0.003165,0.066456,0.00000,0.079114,...,0.148734,0.006329,0.189873,0.012658,0.120253,0.006329,0.018987,0.012658,0.015823,0.117089
1,0.003165,0.009494,0.003165,0.003165,0.006329,0.006329,0.066456,0.000000,0.06962,0.000000,...,0.009494,0.009494,0.018987,0.003165,0.003165,0.117089,0.098101,0.085443,0.091772,0.018987
2,0.003165,0.003165,0.009494,0.037975,0.003165,0.000000,0.000000,0.000000,0.00000,0.006329,...,0.018987,0.164557,0.012658,0.126582,0.012658,0.009494,0.009494,0.015823,0.012658,0.012658
포착군집,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.00000,0.000000,...,0.000000,2.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [29]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))
eval_three_original

,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01
0,0.043062,0.043062,0.052632,0.004785,0.038278,0.023923,0.009569,0.014354,0.009569,0.043062,...,0.119617,0.019139,0.086124,0.033493,0.100478,0.019139,0.023923,0.014354,0.014354,0.062201
1,0.019139,0.043062,0.004785,0.014354,0.019139,0.019139,0.014354,0.004785,0.043062,0.009569,...,0.014354,0.009569,0.023923,0.009569,0.019139,0.105263,0.100478,0.071770,0.057416,0.009569
2,0.009569,0.004785,0.033493,0.052632,0.004785,0.004785,0.004785,0.009569,0.004785,0.019139,...,0.019139,0.095694,0.014354,0.095694,0.023923,0.019139,0.019139,0.023923,0.014354,0.004785
포착군집,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,2.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [30]:
evaluate.calculate_catch_probability(eval_one, eval_two, eval_three)

0.7660182848162885

In [31]:
evaluate.calculate_match_probability(eval_one, eval_two, eval_three)

0.9459459459459459

In [32]:
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, eval_three_original)

0.09165708054954304

In [33]:
dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)#.sort_values("2013-01", axis=1, ascending=False)#.to_csv("2호_4_Q_거리.csv")
dist2 = dist.mean().sort_values(ascending=True)
dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))
dist2.sort_values(ascending=False)

당기순이익/총자본                                                                                                                         0.377155
영업이익/총자본                                                                                                                          0.371833
총자본                                                                                                                               0.208454
영업현금흐름/총자본                                                                                                                        0.182605
매출총이익/총자본                                                                                                                         0.176306
판매 및 관리비/총자본                                                                                                                      0.174686
최대주주비율                                                                                                                            0.155726
차입금의존도                     

In [34]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.714286,0.75,0.750000,0.057143,0.909091,0.973684,0.023810,0.930233,0.085106,0.860465,...,0.838384,0.129412,0.827586,0.035294,0.038462,0.112676,0.138462,0.717949,0.800000,0.5
1,0.160714,0.25,0.027778,0.057143,0.090909,0.000000,0.952381,0.046512,0.851064,0.069767,...,0.101010,0.058824,0.080460,0.835294,0.846154,0.802817,0.753846,0.102564,0.066667,0.5
2,0.125000,0.00,0.222222,0.885714,0.000000,0.026316,0.023810,0.023256,0.063830,0.069767,...,0.060606,0.811765,0.091954,0.129412,0.115385,0.084507,0.107692,0.179487,0.133333,0.0


In [35]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.714286,0.75,0.750000,0.057143,0.909091,0.973684,0.023810,0.930233,0.085106,0.860465,...,0.838384,0.129412,0.827586,0.035294,0.038462,0.112676,0.138462,0.717949,0.800000,0.5
1,0.160714,0.25,0.027778,0.057143,0.090909,0.000000,0.952381,0.046512,0.851064,0.069767,...,0.101010,0.058824,0.080460,0.835294,0.846154,0.802817,0.753846,0.102564,0.066667,0.5
2,0.125000,0.00,0.222222,0.885714,0.000000,0.026316,0.023810,0.023256,0.063830,0.069767,...,0.060606,0.811765,0.091954,0.129412,0.115385,0.084507,0.107692,0.179487,0.133333,0.0


In [36]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [37]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,-0.293086,0.046468,0.050840,1,2.5
A000040,0.279627,0.486258,0.396640,0,2.5
A000050,-0.101406,-0.567613,0.366557,2,2.5
A000070,-0.567732,-0.098500,0.305568,1,2.5
A000080,-0.715476,0.484188,0.323525,1,2.5
...,...,...,...,...,...
A353810,-0.699793,0.746768,-0.296398,1,2.5
A354200,0.843447,-0.267037,-0.181768,0,2.5
A357780,-0.893435,-0.237367,-0.275114,1,2.5
A363280,-0.097304,-0.863496,0.129423,2,2.5
